In [70]:
import os
import pandas as pd

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

from buildingspy.io.outputfile import Reader
from buildingspy.io.postprocess import Plotter

In [71]:
reader = Reader('HeatingCoilValve_ValidationWithB33Data.mat', "dymola")
df = pd.DataFrame()
df["time"] = list(reader.values("heaValSta_F.TSup")[0])
df["mea_tsup"] = list(reader.values("heaValSta_F.TSup")[1])
df["tsup_set"] = list(reader.values("heaValSta_F.TSupSet")[1])
df["valve_trend"] = list(reader.values("percConvHeaValSig.y")[1])
df["cont_sig_modelica_alc"] = list(reader.values("heaValSta_F.yHeaVal")[1])

In [72]:
# list of even numbers range 0 - df.shape[0]
even_index = range(0, df.shape[0], 2)
df_at_change = df.loc[even_index, :].reset_index(drop = True)

In [73]:
def alc_controller(mea_T, set_T, int_error_prev_timestep, kp, ki):
    prop = kp * (set_T - mea_T)
    int_adder_this_timestep = ki * (set_T - mea_T)
    inte = int_error_prev_timestep + int_adder_this_timestep
    
    sig = min(1, prop + inte)
    sig = max(sig, 0)
    
    return inte, sig
    


In [74]:
df_at_change["inte"] = 0
df_at_change["sig"] = .5

In [75]:
for i in range(1, df_at_change.shape[0]):
    mea_t = df_at_change.loc[i, "mea_tsup"]
    set_t = df_at_change.loc[i, "tsup_set"]
    int_error_prev_timestep = df_at_change.loc[i - 1, "inte"]
    inte, sig = alc_controller(mea_t, set_t, int_error_prev_timestep, .05, .005)
    
    df_at_change.loc[i, "inte"] = inte
    df_at_change.loc[i, "sig"] = sig
    

In [77]:
df_test = df_at_change.loc[0:200, :]
df_test

,time,mea_tsup,tsup_set,valve_trend,cont_sig_modelica_alc,inte,sig
0,7905077.0,65.699997,66.0,0.435,0.0150,0.000000,0.500000
1,7905092.0,65.699997,66.0,0.435,0.0150,0.001500,0.016500
2,7905107.0,65.699997,66.0,0.435,0.0165,0.003000,0.018000
3,7905122.0,65.699997,66.0,0.435,0.0600,0.004500,0.019500
4,7905137.0,65.699997,66.0,0.435,0.0615,0.006000,0.021000
5,7905152.0,65.699997,66.0,0.435,0.1050,0.007500,0.022500
6,7905167.0,65.699997,66.0,0.435,0.1065,0.009000,0.024000
7,7905182.0,65.699997,66.0,0.435,0.1500,0.010500,0.025500
8,7905197.0,65.699997,66.0,0.435,0.1515,0.012000,0.027000
9,7905212.0,65.699997,66.0,0.435,0.1950,0.013500,0.028500


In [50]:
df_test.round(3).to_csv('test.csv')

In [78]:
df.head(200)

,time,mea_tsup,tsup_set,valve_trend,cont_sig_modelica_alc
0,7905077.0,65.699997,66.0,0.435,0.0150
1,7905092.0,65.699997,66.0,0.435,0.0150
2,7905092.0,65.699997,66.0,0.435,0.0150
3,7905107.0,65.699997,66.0,0.435,0.0165
4,7905107.0,65.699997,66.0,0.435,0.0165
5,7905122.0,65.699997,66.0,0.435,0.0600
6,7905122.0,65.699997,66.0,0.435,0.0600
7,7905137.0,65.699997,66.0,0.435,0.0615
8,7905137.0,65.699997,66.0,0.435,0.0615
9,7905152.0,65.699997,66.0,0.435,0.1050
